# Выбор метода бинарной классификации на полях `Amount` и `Value`

In [1]:
import numpy  as np
import pandas as pd

In [2]:
df_trn    = pd.read_csv('../data/training_le.csv')
df_tst    = pd.read_csv('../data/test_le.csv')
df_sbm    = pd.read_csv('../data/sample_submission.csv')

df_trn_sc = pd.read_csv('../data/training_le.csv')
df_tst_sc = pd.read_csv('../data/test_le.csv')

In [3]:
df_trn.columns

Index(['BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId',
       'ProductId', 'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

In [4]:
columns4drop = [
    'BatchId', 
    'AccountId', 
    'SubscriptionId', 
    'CustomerId', 
    'ProviderId',
    'ProductId',
    'ProductCategory',
    'ChannelId',
    'TransactionStartTime',
    'PricingStrategy']

In [5]:
df_trn    = df_trn.drop(columns4drop, axis=1)
df_tst    = df_tst.drop(columns4drop, axis=1)

df_trn_sc = df_trn_sc.drop(columns4drop, axis=1)
df_tst_sc = df_tst_sc.drop(columns4drop, axis=1)

**Scaling**

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

In [ ]:
columns4MinMaxScaler = [
    'BatchId',
    'AccountId', 
    'SubscriptionId', 
    'CustomerId', 
    'ProviderId',
    'ProductId', 
    'ProductCategory', 
    'ChannelId',
    'PricingStrategy'
]

In [ ]:
# int --> float
for clm in columns4MinMaxScaler:
    df_trn_sc[clm] = df_trn_sc[clm].astype(float)
    df_tst_sc[clm] = df_tst_sc[clm].astype(float)

In [7]:
def scaleColumns(data, cols_to_scale, scaler):
    for col in cols_to_scale:
        data[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data[col])))
    return data

In [ ]:
df_trn_sc = scaleColumns(df_trn_sc, columns4MinMaxScaler, MinMaxScaler())
df_tst_sc = scaleColumns(df_tst_sc, columns4MinMaxScaler, MinMaxScaler())

In [8]:
df_trn_sc = scaleColumns(df_trn_sc, ['Amount', 'Value'], RobustScaler())
df_tst_sc = scaleColumns(df_tst_sc, ['Amount', 'Value'], RobustScaler())

In [9]:
df_trn.head()

,Amount,Value,FraudResult
0,1000.0,1000,0
1,-20.0,20,0
2,500.0,500,0
3,20000.0,21800,0
4,-644.0,644,0


In [10]:
df_trn_sc.head()

,Amount,Value,FraudResult
0,0.000000,0.000000,0
1,-0.357895,-0.207407,0
2,-0.175439,-0.105820,0
3,6.666667,4.402116,0
4,-0.576842,-0.075344,0


In [11]:
X    = df_trn.drop('FraudResult', axis=1)
y    = df_trn['FraudResult']

X_sc = df_trn_sc.drop('FraudResult', axis=1)
y_sc = df_trn_sc['FraudResult']

In [12]:
from sklearn.model_selection import train_test_split

In [17]:
# This is explicitly used for undersampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

X_sc_train, X_sc_test, y_sc_train, y_sc_test = train_test_split(X_sc, y_sc, test_size=0.2, random_state=24)

In [18]:
# Turn the values into an array for feeding the classification algorithms.
X_train = X_train.values
X_test  = X_test.values
y_train = y_train.values
y_test  = y_test.values

X_sc_train = X_sc_train.values
X_sc_test  = X_sc_test.values
y_sc_train = y_sc_train.values
y_sc_test  = y_sc_test.values

In [19]:
from collections import Counter
print('Train Label Distribution: {}'.format(Counter(y_train)))
print('Train-Scaled Label Distribution: {}'.format(Counter(y_sc_train)))

Train Label Distribution: Counter({0: 76380, 1: 149})
Train-Scaled Label Distribution: Counter({0: 76380, 1: 149})


In [ ]:
# Classifier Libraries
from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.neural_network import MLPClassifier

In [ ]:
classifiers = {
    'Naive Bayes                       ': GaussianNB(),
    'k-Nearest Neighbors               ': KNeighborsClassifier(3),
    'Decision Tree Classifier          ': DecisionTreeClassifier(max_depth=5),
    'Logisitic Regression              ': LogisticRegression(),
    'AdaBoost Classifier               ': AdaBoostClassifier(),
    'Bagging Classifier                ': BaggingClassifier(),
    'Extra-Trees Classifier            ': ExtraTreesClassifier(),
    'Gradient Boosting                 ': GradientBoostingClassifier(),
    'Random Forest                     ': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    'Histogram-based GradBoostClassTree': HistGradientBoostingClassifier(),
    'Linear Discriminant Analysis      ': LinearDiscriminantAnalysis(),
    'Quadratic Discriminant Analysis   ': QuadraticDiscriminantAnalysis(),
    'Multilayer Perceptron             ': MLPClassifier(alpha=1, max_iter=1000)
}
#     'Voting Classifier              ': VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft'),

In [ ]:
from time import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

In [ ]:
# iterate over classifiers
print('Classifiers \t\t Cross_val_score \t F1-score \t Time')
for name, classifier in classifiers.items():
    f1 = []
    tac = time()
    classifier.fit(X_train, y_train)
#     training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    
    predict = classifier.predict(X_test)
    f1.append(f1_score(y_test, predict))
    tic = time()
    print(name, round(training_score.mean(), 5), ' \t', round(np.mean(f1), 5), '\t', round(tic-tac, 2))

In [ ]:
rf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
print(cross_val_score(rf, X_train, y_train, cv=5))  

In [ ]:
classifiers_sc = {
    'Support Vector Classifier, Linear': SVC(kernel="linear", C=0.025),
    'Support Vector Classifier, RBF   ': SVC(kernel="rbf", gamma=2, C=1),
}

In [ ]:
# iterate over classifiers_sc
print('Classifiers \t\t Cross_val_score \t Time')
for name, classifier in classifiers_sc.items():
    tac = time()
    classifier.fit(X_sc_train, y_sc_train)
    training_score = cross_val_score(classifier, X_sc_train, y_sc_train, cv=5)
    tic = time()
    print(name, round(training_score.mean(), 5), ' \t', round(tic-tac, 2))

### Submission - AdaBoost

In [ ]:
classifier = AdaBoostClassifier()
classifier.fit(X_train, y_train)
predict = classifier.predict(X_test)
print(f1_score(y_test, predict))

In [ ]:
FraudPred_AB = classifier.predict(X_test)

In [ ]:
FraudPred_AB = [ 1 if x == -1 else 0 for x in FraudPred_AB ]

In [ ]:
df_sbm['FraudResult'] = FraudPred_AB

In [ ]:
print('Test Label Distribution: {}'.format(Counter(df_sbm['FraudResult'])))

In [ ]:
df_sbm.to_csv('../submitted/AlBo_07_11_AdaBoost.csv', encoding='utf-8', index=False)

### Submission - Decision Tree

In [ ]:
classifier = DecisionTreeClassifier(max_depth=5)
classifier.fit(X_train, y_train)
predict = classifier.predict(X_test)
print(f1_score(y_test, predict))

In [ ]:
X_test = df_tst.values

In [ ]:
FraudPred_AB = classifier.predict(X_test)

In [ ]:
FraudPred_AB = [ 1 if x == -1 else 0 for x in FraudPred_AB ]

In [ ]:
df_sbm['FraudResult'] = FraudPred_AB

In [ ]:
print('Test Label Distribution: {}'.format(Counter(df_sbm['FraudResult'])))

In [ ]:
df_sbm.to_csv('../submitted/AlBo_07_11_AdaBoost.csv', encoding='utf-8', index=False)

# Undersample

In [ ]:
# We will undersample during cross validating
undersample_X = df.drop('Class', axis=1)
undersample_y = df['Class']

for train_index, test_index in sss.split(undersample_X, undersample_y):
    print("Train:", train_index, "Test:", test_index)
    undersample_Xtrain, undersample_Xtest = undersample_X.iloc[train_index], undersample_X.iloc[test_index]
    undersample_ytrain, undersample_ytest = undersample_y.iloc[train_index], undersample_y.iloc[test_index]
    
undersample_Xtrain = undersample_Xtrain.values
undersample_Xtest = undersample_Xtest.values
undersample_ytrain = undersample_ytrain.values
undersample_ytest = undersample_ytest.values 

undersample_accuracy = []
undersample_precision = []
undersample_recall = []
undersample_f1 = []
undersample_auc = []

# Implementing NearMiss Technique 
# Distribution of NearMiss (Just to see how it distributes the labels we won't use these variables)
X_nearmiss, y_nearmiss = NearMiss().fit_sample(undersample_X.values, undersample_y.values)
print('NearMiss Label Distribution: {}'.format(Counter(y_nearmiss)))
# Cross Validating the right way

for train, test in sss.split(undersample_Xtrain, undersample_ytrain):
    undersample_pipeline = imbalanced_make_pipeline(NearMiss(sampling_strategy='majority'), log_reg) # SMOTE happens during Cross Validation not before..
    undersample_model = undersample_pipeline.fit(undersample_Xtrain[train], undersample_ytrain[train])
    undersample_prediction = undersample_model.predict(undersample_Xtrain[test])
    
    undersample_accuracy.append(undersample_pipeline.score(original_Xtrain[test], original_ytrain[test]))
    undersample_precision.append(precision_score(original_ytrain[test], undersample_prediction))
    undersample_recall.append(recall_score(original_ytrain[test], undersample_prediction))
    undersample_f1.append(f1_score(original_ytrain[test], undersample_prediction))
    undersample_auc.append(roc_auc_score(original_ytrain[test], undersample_prediction))